# Unfolding of an Experimental 16 MeV on Be PHS: 0.5 MeV bins
This notebook contains the specific commands used to take the 16 MeV on Be experimental data from Root TTrees and turn them into the correctly formatted .phs and .rsp files for the the NSD and HEPROW unfolding programs.  It will also generate the input neutron spectrum to use in comparison.  

Root must be installed and PyRoot enabled for this notebook to work.

First, load the necessary libraries, set the appropriate environment variables, and compile the C++ support macros from the instructions @ https://bitbucket.org/berkeleylab/nsd-rootscripts/wiki/LibraryCompilation.

Load python dependencies:

In [1]:
from ROOT import gROOT
import numpy as np

gROOT.ProcessLine('HistogramWriter writer;')
gROOT.ProcessLine('HistogramOperations ops;')

47127012139056L

## Specify the user inputs

#### Input File Locations
dataPath = the absolute location of where the root files are located <br/>
phsName = file name for the pulse height spectra TTree <br/>
rspName = file name for the response matrix TTree <br/>

In [2]:
dataPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/"
phsName="Experiments/PHS/16MeVBe/SinglePh.root"

rspName="Simulated/PHS/ResponseMatrices/simSideResponse100Mil.root"

#### Output File Location
outPath = the absolute path of where to place the generated HEPROW input files

In [3]:
outPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Experiments/PHS/16MeVBe/Unfold/HEPROW/Inputs/"

#### Response Matrix Binning

rspEwidth = the width of bins to use for the binning of the response matrix energy <br/>
rspEmin = the minimum Energy to use for the binning of the response matrix  <br/>
rspEmax = the maximum Energy to use for the binning of the response matrix  <br/>
rspLwidth = the width of bins to use for the binning of the response matrix light <br/>
rspLmin = the minimum Light to use for the binning of the response matrix  <br/>
rspLmax = the maximum Light to use for the binning of the response matrix  <br/>

In [4]:
rspEwidth=0.5
rspEmin=0.0
rspEmax=20.0
rspLwidth=0.05
rspLmin=0.0
rspLmax=14.0

## Create the .phs File
### Load PHS Data File

In [ ]:
gROOT.ProcessLine('TFile f("{}")'.format(dataPath+phsName))

### Apply Calibration and print the .phs file

The calibration constants used were:

3)
a = 0.00327102  <br/>
b = -0.427562 <br/>
Ec = 0.0531585  <br/>
E1 = 0.0310429 <br/>
E2 = 0.0151981 <br/>

In [ ]:
gROOT.ProcessLine('ops.applyCalibration(histo3, 0.00327102, -0.427562);')

In [ ]:
gROOT.ProcessLine('histo3->Draw()')

Truncate all negative and all zeros data until there is n * rebin bins left remaining.  This makes the bottom of the PHS cleaner and easier for working with in HEPROW.  

Find the first bin with data.  The cell will print the first bin, but not the lower MeVee edge.  It is a bit clunky, but you can read it directly from the terminal running this notebook (or run the commands in Root directly).

In [ ]:
gROOT.ProcessLine('tmpSpectrum = (TH1D*)ops.truncateHist(histo3,0.105,14);')

i=0
while gROOT.ProcessLine('tmpSpectrum->GetBinContent({})'.format(i))==0:
    i+=1
print i

gROOT.ProcessLine('tmpSpectrum->GetXaxis()->GetBinLowEdge({})'.format(i))

Choose a rebin number to get the desired MeVee/bin.  In this case, 16 was chosen to get ~ 0.05 MeVee/bin 

In [ ]:
rebin=16
gROOT.ProcessLine('tmpSpectrum->Rebin({})'.format(rebin))
gROOT.ProcessLine('PulseHeightSpectrum = (TH1D*)ops.truncateHist(tmpSpectrum,0.31,14);')
gROOT.ProcessLine('PulseHeightSpectrum->GetXaxis()->GetBinLowEdge({})'.format(0))

Print out the .phs file for both statistic bins and the intrinsic rebin from above:

In [ ]:
gROOT.ProcessLine('TH1* dataHist = ops.rebinStatistically(PulseHeightSpectrum,100);')
gROOT.ProcessLine('writer.PhToHEPROW(PulseHeightSpectrum,"{}histo3_phs_05")'.format(outPath))
gROOT.ProcessLine('writer.PhToHEPROW(dataHist,"{}histo3_stat_100_phs_05")'.format(outPath))

## Ouput the Initial Spectrum

Uses the simulated 16 MeV spectrum, which was created from Meulders

In [ ]:
gROOT.ProcessLine('TFile g("/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Simulated/PHS/16MeVBe/meulders16BeSimulation.root)')
gROOT.ProcessLine('primary->Draw("nEn>>initSpec(20,0,20)")')
gROOT.ProcessLine('writer.Th1ToAscii(initSpec,"{}inputSpectrum_1")'.format(outPath))

## Create the .rsp File
### Load Response Matrix Data File

In [5]:
gROOT.ProcessLine('SimulationManipulation sm("{}",0)'.format(dataPath+rspName))

47127012139104L

### Create the bin structures and print .rsp file

In [6]:
# Create the bin structures
rspEbins=np.arange(rspEmin,rspEmax,rspEwidth)
rspEbins=np.append(rspEbins,rspEmax)
#print rspEbins
rspLbins=np.arange(rspLmin,rspLmax,rspLwidth)
rspLbins=np.append(rspLbins,rspLmax)
#print rspLbins
gROOT.ProcessLine('const Int_t EBINS = {}; const Int_t LBINS = {};'.format(len(rspEbins)-1,len(rspLbins)-1))
gROOT.ProcessLine('Double_t eEdges[EBINS + 1] = {}{}{};'.format("{",", ".join(str(e) for e in rspEbins),"}"))
gROOT.ProcessLine('Double_t lEdges[LBINS + 1] = {}{}{};'.format("{",", ".join(str(e) for e in rspLbins),"}"))
gROOT.ProcessLine('axis1 = TAxis(EBINS,eEdges);')
gROOT.ProcessLine('axis2 = TAxis(LBINS,lEdges);')

# Create the Histogram and output file
gROOT.ProcessLine('TH2* hist=sm.getNormalizedResponseMatrix(axis1,axis2)')
gROOT.ProcessLine('hist->Draw("colz")')
gROOT.ProcessLine('writer.ResponseToHEPROW(hist,"{}resp_05_50")'.format(outPath))

0L

### Smear the Response Matrix and Create the .rsp File

In [7]:
gROOT.ProcessLine('TH2* smearHist2 = ops.skewedGausSmearMatrix(hist,0.0531585, 0.0310429, 0.0151981)')
gROOT.ProcessLine('smearHist2->Draw("colz")')
gROOT.ProcessLine('writer.ResponseToHEPROW(smearHist2,"{}smearedResp_05_50")'.format(outPath))

0L

## Mik.inp Support

These lines are useful to develop the mik.inp file:

In [9]:
print "{}".format(len(rspEbins)-2)
for i in range(1,len(rspEbins)-1):
    print "{:.1f}                                     right boundary of {:1d} energy interval".format(rspEbins[i],i)

39
0.5                                     right boundary of 1 energy interval
1.0                                     right boundary of 2 energy interval
1.5                                     right boundary of 3 energy interval
2.0                                     right boundary of 4 energy interval
2.5                                     right boundary of 5 energy interval
3.0                                     right boundary of 6 energy interval
3.5                                     right boundary of 7 energy interval
4.0                                     right boundary of 8 energy interval
4.5                                     right boundary of 9 energy interval
5.0                                     right boundary of 10 energy interval
5.5                                     right boundary of 11 energy interval
6.0                                     right boundary of 12 energy interval
6.5                                     right boundary of 13 energy interval
7.0  

## NSD Unfold

This uses the unfolding algorithms developed by Matt Harasty and Josh Brown contained in the nsd-rootscripts library to unfold the same data.  

First we must load the script to do the unfolding. NOTE: All of the parameters set in this notebook above are done explicitly within this script.  It is recommended that a unique script be made in each unfold directory to document the settings used.

NOTE: Do not run the ROOT import cell if it was run about and still active in the kernel.  It is repeated here for convience when not doing the HEPROW processing.

In [ ]:
from ROOT import gROOT

gROOT.ProcessLine('HistogramWriter writer;')

In [ ]:
gROOT.ProcessLine('.L /home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Experiments/16MeVBe/Unfold/16MeVBeUnfold.cpp')

### Run the Algorithm
Create the unfolding object, run the unfolding algorithm, and plot the results:

In [ ]:
gROOT.ProcessLine('NeutronSpectrumUnfolding* obj = setupHalfMeVUnfolding()')
gROOT.ProcessLine('obj->runRootMinimization("Minuit")')
gROOT.ProcessLine('obj->drawState()')

### Write the results 

Save to both a root file and .phs file:

In [ ]:
gROOT.ProcessLine('obj->writeState("/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Experiments/16MeVBe/Unfold/NSDUnfolded16MeV_Cal3_50.root")')
gROOT.ProcessLine('TH1D* outputHisto = obj->getCurrentTrialClone()')
gROOT.ProcessLine('writer.Th1ToAscii(outputHisto, "/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Experiments/16MeVBe/Unfold/NSD16MeVSpectrum_Cal3_0")')